In [5]:
import os
import tikzplotlib
import pandas as pd
import matplotlib.pyplot as plt
import Utilities.DataProcessing.data_import as dp_utils
import Utilities.Plotting.plot_distributions_spectra as plt_dist
import Utilities.Plotting.plotting_utilities as plt_utils
import Utilities.DataProcessing.signal_processing_utils as sigutils

In [6]:
if __name__ == "__main__":
    fig_save_path = "./Figures"
    filename = "P2017_20_Solarhouse_2.csv"
    hybridcosim_path = os.environ.get("HYBRIDCOSIM_REPO_PATH", "../../")
    path_to_dir = os.path.join(hybridcosim_path, "Data/AEE/")
    path_full = os.path.join(path_to_dir, filename)
    datahouse = pd.read_csv(path_full, sep=";")

    lab = ['T_P_oo', 'T_P_o', 'T_P_mo', 'T_P_mu', 'T_P_u', 'T_Holzkessel', 'T_Wohnraumofen',
           'T_Solar_VL', 'T_Solar_RL', 'Vd_Solar',
           'T_Nachheizung_VL', 'T_Nachheizung_RL', 'Vd_Nachheizung',
           'T_WW_VL', 'T_WW_RL', 'Vd_WW',
           'T_FBH_VL', 'T_FBH_RL', 'Vd_FBH',
           'Qel_Technik', 'Pel_Technik', 'Qel_Haushalt', 'Pel_Haushalt',
           'T_Raum', 'P_Holzkessel_calc', 'P_Ofen_calc', 'P_Recool']

    N_samples = 5000
    start = 375000 + 700
    start = 0
    df_orig = dp_utils.parse_hdf_solarhouse2(path_full)
    df = df_orig.iloc[start:start+N_samples]

    colors = ['red', 'blue', 'orange', 'purple']

HDF5ExtError: HDF5 error back trace

  File "C:\ci\hdf5_1611496732392\work\src\H5F.c", line 793, in H5Fopen
    unable to open file
  File "C:\ci\hdf5_1611496732392\work\src\H5VLcallback.c", line 3500, in H5VL_file_open
    open failed
  File "C:\ci\hdf5_1611496732392\work\src\H5VLcallback.c", line 3465, in H5VL__file_open
    open failed
  File "C:\ci\hdf5_1611496732392\work\src\H5VLnative_file.c", line 100, in H5VL__native_file_open
    unable to open file
  File "C:\ci\hdf5_1611496732392\work\src\H5Fint.c", line 1707, in H5F_open
    unable to read superblock
  File "C:\ci\hdf5_1611496732392\work\src\H5Fsuper.c", line 412, in H5F__super_read
    file signature not found

End of HDF5 error back trace

Unable to open/create file 'D:\GitHub\TUG-CPS\hybridcosim\Data/AEE/P2017_20_Solarhouse_2.csv'

In [ ]:
    '''Storage Tank Labels'''
    storage_tank_labels = ['T_P_top', 'T_P_o', 'T_P_mo', 'T_P_mu', 'T_P_u']
    storage_tank_legends = ['Storage top top', 'Storage top', 'Storage middle top', 'Storage middle bottom', 'Storage bottom' ]

    '''Temperature Labels'''
    temperature_labels = ['T_Solar_VL', 'T_WW_RL', 'T_FBH_RL', 'T_Nachheizung_VL']
    temperature_legends = ['Solar supply', 'Domestic hot water return', 'Floor Heating return', 'Stove/Boiler supply']

    ''' Solar labels'''
    solar_labels = ['T_Solar_VL', 'T_Solar_RL', 'T_Solar_pri_VL','T_Solar_pri_RL']
    solar_colors = ['red', 'blue', 'red', 'blue']
    linestyles = ['-', '-', '--','--']
    solar_legends = ['Solar supply', 'Solar return', 'Solar Supply Primary (collector side)', 'Solar return primary']

    ''' Y Labels '''
    ylabel_temperature = 'Temperature [°C]'
    ylabel_massflow = 'Volume flow rate'

    '''Storage Tank Temperatures'''

    # normalize mass flows
    massflow_labels = ['Vd_Solar', 'Vd_WW', 'Vd_FBH', 'Vd_Nachheizung']

In [ ]:
    for label in massflow_labels:
        sigutils.scale_curve(df, label)

    scaled_massflow_labels = ['Vd_Solar_scaled', 'Vd_FBH_scaled', 'Vd_WW_scaled', 'Vd_Nachheizung_scaled']
    scaled_massflow_legends = ['Scaled Solar Volume flow', 'Scaled Warm Water Volume flow',
                               'Scaled Floor Heating Volume flow', 'Scaled Boiler Volume flow']

In [ ]:
    '''Floor Heating'''
    fbh_temperature_labels = ['T_FBH_VL', 'T_FBH_RL']
    fbh_temperature_legends = ['Floor Heating Supply', 'Floor Heating Return']

    data = [df[storage_tank_labels], df[fbh_temperature_labels], [df[['Vd_FBH']],df[['Pu_FBH']]]]
    labels = [storage_tank_legends, fbh_temperature_legends, [['Volume flow Floor Heating'],['Pump Floor Heating']]]
    plt_utils.plt_subplots(data,fig_save_path=fig_save_path, fig_title='Floor Heating', linestyle="--", figsize=(10,10),show_ylabel=False)


In [ ]:
    # Plot spectrum of floor heating
    #df = df.iloc[:N_samples]
    signal = df['Vd_FBH']
    time_axis = df.index
    signal_zeromean, offset = sigutils.remove_offset(signal)
    fft_data = sigutils.fft_abs(signal_zeromean)
    plt_dist.plot_spectra(signal_zeromean, fft_data=fft_data)

In [ ]:
    # Main frequency: Period: 20 samples - Frequency 0.05 fs
    T = 20
    # Cutoff frequency: 1 / 10 * main frequency - 0.005 fs
    y = sigutils.filter_signal(signal, T)
    env_avg_filtered, envelope_h, envelope_l = sigutils.calc_avg_envelope(signal, T)

    plt.figure(figsize=(20,10))
    #plt.plot(signal, linestyle = '--', label='Original signal')
    plt.plot(time_axis, y, label='Filtered signal')
    plt.plot(time_axis, envelope_h, label='Envelope high', color='gray')
    plt.plot(time_axis, envelope_l, label='Envelope low', color='gray')
    plt.plot(time_axis, env_avg_filtered, label='Envelope average filtered')
    plt.title('VD_FBH')
    plt.legend()
    plt.show()

In [ ]:
    signal_zeromean, _ = sigutils.remove_offset(env_avg_filtered)
    fft_data = sigutils.fft_abs(signal_zeromean, spectrum_range=0.05)
    plt_dist.plot_spectra(signal_zeromean, fft_data)

In [ ]:
    signal_zeromean,_ = sigutils.remove_offset(y)
    fft_data = sigutils.fft_abs(signal_zeromean, spectrum_range=0.05)
    plt_dist.plot_spectra(signal_zeromean, fft_data)


In [ ]:
    signal = df['T_FBH_VL']
    signal_zeromean, _ = sigutils.remove_offset(signal)

    df['T_FBH_VL_filtered'] = y
    df['T_FBH_VL_env_h'] = envelope_h
    df['T_FBH_VL_env_l'] = envelope_l
    df['T_FBH_VL_env_avg_filtered'] = env_avg_filtered

    labels_envs = ['T_FBH_VL_env_h', 'T_FBH_VL_filtered',  'T_FBH_VL_env_l', 'T_FBH_VL_env_avg_filtered']
    legends_envs = ['Envelope high ','Filtered Signal', 'Envelope low', 'Envelope average filtered']

    data = [df[labels_envs]]
    labels = [labels_envs]
    plt_utils.plt_subplots(data,fig_save_path=fig_save_path, fig_title='TFBH_VL', linestyle="--", figsize=(10,10),show_ylabel=False)

In [ ]:
    env_thresholded, _ = sigutils.threshold_data(df, labels_envs, 'Pu_FBH')
    labels_env_thresholded = [f"{label}_thresh" for label in labels_envs]
    legends_env_thresholded = [f"{legend} thresholded" for legend in legends_envs]
    for label in labels_envs:
        df[f"{label}_thresh"] = env_thresholded[label]


    data = [[df[labels_env_thresholded], df[['Pu_FBH']]]]
    labels = [[legends_env_thresholded, 'Pu_FBH']]
    plt_utils.plt_subplots(data,fig_save_path=fig_save_path, fig_title='TFBH_VL - Thresholded',linestyle="--", figsize=(10,10),show_ylabel=False)

    #Vd_thresholded = lowpass_fft(signal,fcut=0.1)
    #plot_spectra(Vd_thresholded)


In [ ]:
    import numpy as np
    percentage = np.sum(df['Vd_Solar'] > 0) / df.shape[0]
    print(percentage)


In [ ]:
    data = [df_orig[['T_Solar_VL','T_Solar_RL']],df[['T_Solar_VL', 'T_Solar_RL']]]
    plt_utils.plt_subplots(data,fig_save_path=fig_save_path, fig_title='T_Solar_VL', linestyle="--", figsize=(10,4),show_ylabel=False)


In [ ]:
    sig_zm,_ = sigutils.remove_offset(df_orig['T_Solar_VL'])
    fft_freqs, fft_abs = sigutils.fft_abs(sig_zm)
    plt.figure(figsize=(10,5))
    plt.title('Spectral Analysis - T_Solar_VL')
    plt.stem(fft_freqs, fft_abs)
    plt.ylabel('Magnitude')
    plt.xlabel('Frequency (normalized)')
    plt.savefig('./Figures/spec_tsolarvl.png')
    tikzplotlib.save('./Figures/spec_tsolarvl.tex')
    plt.show()

In [ ]:
    def get_mean_over_time_ranges(data, time):
        import numpy as np
        return [np.mean(data[curtime[0]:curtime[1]].values) for curtime in time]

    def get_vals_in_hour_range(data, hours):
        return data[data.index.hour.isin(hours)]

    df = df_orig
    import pandas as pd
    from pandas.tseries.offsets import MonthEnd
    time = [[pd.Timestamp(2019,month,1),pd.Timestamp(2019,month,1) + MonthEnd(1)] for month in range(2,12)]
    print(time)
    TSolarVL = df[['T_Solar_VL']]

    import matplotlib.pyplot as plt
    import numpy as np
    hours = list(range(6)) + list(range(18,24))
    # TODO improve this code!
    df['Mean night value'] = np.zeros((df.shape[0]))
    means = get_mean_over_time_ranges(get_vals_in_hour_range(TSolarVL, hours), time)
    for curtime, mean in zip(time, means):
        df['Mean night value'][curtime[0]:curtime[1]] = mean
    print(np.mean(means))
    plt_utils.plt_subplots([TSolarVL[curtime[0]:curtime[1]].join(df[['Mean night value']][curtime[0]:curtime[1]]) for curtime in time],fig_save_path=fig_save_path, fig_title= 'Seasonal Trend', linestyle="--", figsize=(15,20),show_ylabel=False, ylim=(25,65))

In [ ]:
    plt.figure(figsize=(10,2))
    plt.plot(df[['T_Solar_VL', 'T_Solar_RL']][0*96+0:0*96+96])
    plt.ylabel('TSolarVL, TSolarRL')
    plt.legend(['TSolarVL','TSolarRL'])
    plt.show()

In [ ]:
    plt.figure(figsize=(10,2))
    plt.plot(df['T_Solar_VL'][df['Vd_Solar_inv'] <= 0.5 * max(df['Vd_Solar_inv'])])

